# 3.4 - Perguntas sobre os Reviews

In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/06 00:54:49 WARN Utils: Your hostname, MacBook-Air-de-Vitor.local, resolves to a loopback address: 127.0.0.1; using 192.168.3.49 instead (on interface en0)
26/02/06 00:54:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/06 00:54:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/06 00:54:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/06 00:54:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Carregando CSV

In [2]:
path = "data/raw/olist_order_reviews_dataset.csv"

df = spark.read.csv(path, header=True)
df.show(5)

+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                NULL|                  NULL| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                NULL|                  NULL| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                NULL|                  NULL| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                NULL|  Recebi bem antes ...| 2017-04-21 00:00:00|   

In [3]:
mensagem = "review_comment_message"
titulo = "review_comment_title"

## Qual a porcentagem de pessoas que escrevem review

In [4]:
# Denominador
total = df.count()

# numerador (comment not null)
with_message = (
    df
    .where(
        df[mensagem].isNotNull()
    )
).count()

print(f"{with_message/total * 100:.2f}%")

39.44%


## Porcentagem que escreveram alguma coisa (review ou message)

In [5]:
some_message = df.where(
    df[titulo].isNotNull() |
    df[mensagem].isNotNull()
).count()

print(f"{some_message/total * 100:.2f}%")

41.42%


## Porcentagem que escreveram titulo e mensagem

In [9]:
some_message = df.where(
    df[titulo].isNotNull() &
    df[mensagem].isNotNull()
).count()

print(f"{some_message/total * 100:.2f}%")

9.55%


## Porcentagem de reviews bons escritos em maiusculo

In [17]:
bons_reviews = (
    df
    .where(F.length(F.col("review_score"))==1)
    .where(F.col("review_score")>=4)
)

denominador = bons_reviews.count()

numerador = (
    bons_reviews
    .withColumn("upper", F.upper(F.col(mensagem)))
    .where(F.col(mensagem) == F.col("upper"))
).count()

print(f"{numerador/denominador * 100:.2f}%")

1.30%


In [18]:
contem = (
    bons_reviews
    .where(F.col('review_comment_message').like('%recomendo%'))
).count()


termina = (
    bons_reviews
    .where(F.col('review_comment_message').like('%recomendo'))
).count()

starswith = (
    bons_reviews
    .where(F.col('review_comment_message').like('recomendo%'))
).count()

print(
    round(contem/denominador*100, 2), 
    round(termina/denominador*100, 2), 
    round(starswith/denominador*100, 2),
    sep='\n',
)

2.7
0.53
0.18
